ITESM CCM
Oscar Fuentes


Para correr este notebook debe tenerse familiaridad con términos de ROS como tópicos, listeners, publishers, mensajes etc.


In [1]:
import math
import numpy as np
import rospy
import time
from sensor_msgs.msg import LaserScan

from geometry_msgs.msg import Twist


Introducimos ahora dos nuevos tipos de mensajes relevantes. Imagen  y nube de puntos, Ambos mensajes propios de ros, 
se muestran algunas librerías que hacen más fácil su utilización en Python
https://www.ros.org/
http://wiki.ros.org/ros_numpy
http://wiki.ros.org/cv_bridge


In [2]:
import ros_numpy



En este notebook una simple maquina de estados para evasión de obstáculos

In [3]:
rospy.init_node('evasion_notebook_node') 
base_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

Nos valdremos de nuestra vieja función de movimiento, asi como la clase Laser para obtener las lecturas del sensor (revisar notebook 1)

Simultáneamente , es publicado en el tópico "/hsrb/head_rgbd_sensor/depth_registered/rectified_points"
, información de la nube de puntos, en forma de un mensaje tipo  PointCloud2

In [4]:

def move_base_vel(vx, vy, vw):
    twist = Twist()
    twist.linear.x = vx
    twist.linear.y = vy
    twist.angular.z = vw 
    base_vel_pub.publish(twist)

def move_base(x,y,yaw,timeout=5):
    start_time = rospy.Time.now().to_sec()
    while rospy.Time.now().to_sec() - start_time < timeout:  
        move_base_vel(x, y, yaw)
    move_base_vel(0,0, 0)

In [7]:
move_base(1,0,0,1 )

In [15]:
move_base(0,0,1.57,1)

In [10]:
###COMENTAR Y DISCUTIR

![alt text](state_machine.png "Maquina de estados")

In [5]:
def move_forward():
    move_base(0.15,0,0,2.5)
def move_backward():
    move_base(-0.05,0,0,1.5)
def turn_left():
    move_base(0.0,0,0.12*np.pi,2)
def turn_right():
    move_base(0.0,0,-0.12*np.pi,2)

In [9]:
move_forward()

In [10]:
move_backward()

In [11]:
turn_right()

In [12]:
turn_left()

In [5]:
from sensor_msgs.msg import LaserScan

class Laser():
    u"""Class that handles laser information"""

    def __init__(self):
        # Register the _laser_cb method as a callback to the laser scan topic events
        self._laser_sub = rospy.Subscriber ('scan',
                                           LaserScan, self._laser_cb)
        self._scan_data = None

    def _laser_cb (self, msg):
        # Laser scan callback function
        self._scan_data = msg

    def get_data(self):
        u"""Function to get the laser value"""
        return self._scan_data

In [8]:
laser = Laser()  #instanciamos una clase 

In [7]:
lectura=laser.get_data()
lectura=np.asarray(laser.get_data().ranges)
lectura=np.where(lectura>20,0,lectura) #remove infinito

In [ ]:
lectura=laser.get_data()
lectura=np.asarray(laser.get_data().ranges)
lectura=np.where(lectura>20,0,lectura) #remove infinito
front_scan=  lectura[3*len(lectura)/4:5*len(lectura)/4]
right_scan=  lectura[len(lectura)/2:]   
right_scan= right_scan[::-1]
right_scan=right_scan[:len(right_scan)/2]
left_scan=  lectura[:len(lectura)/4] 


#right_scan=  lectura[len(lectura)/2:(len(lectura)/2)+20]
#left_scan =  lectura[len(lectura)/2-20:len(lectura)/2]

In [ ]:
left_scan=  lectura[:len(lectura)/4] 
left_scan

In [ ]:
front_scan.mean(),right_scan.mean(),left_scan.mean()

In [ ]:
def get_lectura_cuant():
    lectura=np.asarray(laser.get_data().ranges)
    
    lectura=np.where(lectura>3,0,lectura) #remove infinito
    front_scan=  lectura[3*len(lectura)/4:5*len(lectura)/4]
    front_scan=  lectura[3*len(lectura)/4:5*len(lectura)/4]
    right_scan=  lectura[len(lectura)/2:]   
    right_scan= right_scan[::-1]
    right_scan=right_scan[:len(right_scan)/8]
    left_scan=  lectura[:len(lectura)/8] 



    sd,si=0,0
    if np.mean(left_scan) > 0:   si=1
    if np.mean(right_scan)> 0:   sd=1
    return si,sd#front scan 


In [ ]:
si,sd=get_lectura_cuant()
si,sd

In [ ]:
move_forward()
#turn_left()

In [ ]:
si,sd=get_lectura_cuant()
next_state= 's_0'
i=0
print (i, next_state, si, sd)

In [ ]:
for i in range (250):## 20 pasos  pueden ser mas
    print ('step',i,'next ', next_state,'sensor izq', si,'sensor der', sd)
#print (next_state)
    if next_state== 's_0':
        print ('im in state ' , next_state)
        si,sd=get_lectura_cuant()
        #ESTADO 0 obtener lectura
        if (si==0 and sd==0): next_state='s_0'
            
        if (si==0 and sd==1): next_state='s_1'
        if (si==1 and sd==0): next_state='s_3'
        if (si==1 and sd==1): next_state='s_5'
        #####Accion
        move_forward()
        



    elif next_state== 's_1':
        print ('im in state 1')

        #####Accion
        move_backward()

        next_state='s_2'


    elif next_state== 's_2':
        print ('im in state 2')
        #####Accion
        turn_left()
        next_state='s_0'


    elif next_state== 's_3':
        print ('im in state 3')
        #####Accion
        move_backward()
        next_state='s_4'


    elif next_state== 's_4':
        print ('im in state 4')
        #####Accion
        turn_right()
        next_state='s_0'

    elif next_state== 's_5':
        print ('im in state 5')

        #####Accion

        move_backward()
        move_backward()
        move_backward()
        next_state='s_6'

    elif next_state== 's_6':
        print ('im in state 6')
        #####Accion
        turn_left()
        next_state='s_7'

    elif next_state== 's_7':
        print ('im in state 7')
        #####Accion
        turn_left()
        next_state='s_8'

    elif next_state== 's_8':
        print ('im in state 8')

        #####Accion
        move_forward()
        next_state='s_9'

    elif next_state== 's_9':
        print ('im in state 9')
        #####Accion
        #move_forward()
        next_state='s_10'

    elif next_state== 's_10':
        print ('im in state 10')
        #####Accion
        turn_right()
        next_state='s_11'
    elif next_state== 's_11':
        print ('im in state 11')

        #####Accion
        turn_right()
        next_state='s_0'






In [ ]:
print ('step, next_state, ssensorizq, sensor derecha',i, next_state, si, sd)

In [ ]:
move_forward()

In [ ]:
right_scan=  lectura[len(lectura)/4:(len(lectura)/2)-20]
left_scan =  lectura[len(lectura)/2:3*len(lectura)/3]
front_scan=  lectura[len(lectura)/2+20:3*len(lectura)/4]


In [ ]:
right_scan.mean()

In [ ]:
left_scan.mean()

In [8]:
laser._laser_cb()

TypeError: _laser_cb() takes exactly 2 arguments (1 given)

# ODOM

In [6]:
import tf
def newOdom (msg):
    global x
    global y
    global th
    
    x=msg.pose.pose.position.x
    y=msg.pose.pose.position.y
    quaternion = (
    msg.pose.pose.orientation.x,
    msg.pose.pose.orientation.y,
    msg.pose.pose.orientation.z,
    msg.pose.pose.orientation.w)
    euler = tf.transformations.euler_from_quaternion(quaternion)
    th=euler[2]


In [7]:
#READ CLICK POINT
from geometry_msgs.msg import PointStamped
def readPoint(punto):
     global xcl , ycl
     
     xcl =punto.point.x
     ycl =punto.point.y

In [8]:
from nav_msgs.msg import Odometry

sub= rospy.Subscriber("/odom",Odometry,newOdom)


In [9]:
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

In [16]:
sub3=rospy.Subscriber("/clicked_point",PointStamped,readPoint)


In [12]:
xcl,ycl

(1.0952262878417969, 0.38174670934677124)

# POTFIELDS

In [10]:
laser= Laser()

In [17]:
data=laser.get_data()
lec=np.asarray(data.ranges)
lec[np.isinf(lec)]=3.5


In [18]:
deltaang=   ( data.angle_max-data.angle_min  )   /len(data.ranges)

In [19]:
deltaang

0.01745330492655436

In [20]:
laserdegs=  np.arange(data.angle_min,data.angle_max,deltaang)

In [21]:
Fx=0
Fy = 0.001
for i,deg in enumerate(laserdegs):
    if i >=360:break 
    
    if (lec[i] <2.61):
        Fx = Fx + (1/lec[i])**2 * np.cos(deg)
        Fy = Fy + (1/lec[i])**2 * np.sin(deg)
         

In [22]:
Fth= np.arctan2(Fy,Fx)+np.pi
print('FxFyFth',Fx,Fy,Fth*180/np.pi)
Fmag= np.linalg.norm((Fx,Fy))

FxFyFth -162.7382919239599 5.0138214370837675 358.23532640458404


In [23]:
xy,xycl=np.array((x,y)) ,   np.array((xcl,ycl))
euclD=np.linalg.norm(xy-xycl)
#print("xrob,yrob, throbot",x,y,th*180/3.1416)
print("xclick,yclick",xcl,ycl,"euclD",euclD)



xclick,yclick 1.0 0.0 euclD 1.0000822763500168


In [24]:
Fatrx =( -x + xcl)/euclD
Fatry =( -y + ycl)/euclD      
Fatrth=np.arctan2(Fatry, Fatrx) 
Fatrth=Fatrth-th
Fmagat= np.linalg.norm((Fatrx,Fatry))
print ('Fatx, Fatry, Fatrth',Fatrx,Fatry,(Fatrth)*180/np.pi )

Fatx, Fatry, Fatrth 1.0000003935329278 -0.00036655017636064526 -0.35186203701561614


In [25]:
Ftotx= Fmag*np.cos(Fth)*.00251   +    Fmagat*np.cos(Fatrth)
Ftoty= Fmag*np.sin(Fth)*.00251    +    Fmagat*np.sin(Fatrth)
Ftotth=np.arctan2(Ftoty,Ftotx)
##################################################################
if ( Ftotth> np.pi ):Ftotth=       -np.pi-    (Ftotth-np.pi)
if (Ftotth < -np.pi):Ftotth= (Ftotth     +2 *np.pi)


In [26]:
print('Ftotxy',Ftotx,Ftoty,Ftotth*180/np.pi)


Ftotxy 1.4084547166239676 -0.01872580709417788 -0.7617188476317114


In [27]:
speed=Twist()
vel=0.4
if( abs(Ftotth) < .15) :#or (np.linalg.norm((Fx,Fy)) < 100):
     speed.linear.x=1.9
     print('lin')
     speed.angular.z=0
else:
         
        if Ftotth < 0:
            if (abs( Ftotth ) < np.pi/2):
                vel=.24
                speed.angular.z=-0.25
                print('open curve')
         
            print('Vang-')
            speed.linear.x=vel
            speed.angular.z=-0.25
        if Ftotth > 0:
            if (abs( Ftotth ) < np.pi/2):
                vel=.24
                print('open curve')
         
            
            print('Vang+')
            speed.linear.x=vel
            speed.angular.z=.25

lin


In [11]:
def get_Rep_Force():
    data=laser.get_data()
    lec=np.asarray(data.ranges)
    lec[np.isinf(lec)]=13.5
    deltaang=   ( data.angle_max-data.angle_min  )   /len(data.ranges)
    laserdegs=  np.arange(data.angle_min,data.angle_max,deltaang)
    Fx=  0.0
    Fy = 0.0
    for i,deg in enumerate(laserdegs):
        if i >=360:break 

        if (lec[i] < 4.9): ###TUNABLE
            Fx = Fx + (1/lec[i])**2 * np.cos(deg)
            Fy = Fy + (1/lec[i])**2 * np.sin(deg)
    Fth= np.arctan2(Fy,(Fx))+np.pi
    #print('FxFyFth',Fx,Fy,Fth*180/np.pi)
    Fmag= np.linalg.norm((Fx,Fy))
    return Fx,Fy,Fmag,Fth
    

In [12]:
def get_Att_Force():
    xy,xycl=np.array((x,y)) ,   np.array((xcl,ycl))
    euclD=np.linalg.norm(xy-xycl)
    #print("xrob,yrob, throbot",x,y,th*180/3.1416)
    #print("xclick,yclick",xcl,ycl,"euclD",euclD)

    Fatrx =( -x + xcl)/euclD
    Fatry =( -y + ycl)/euclD      
    Fatrth=np.arctan2(Fatry, Fatrx) 
    Fatrth=Fatrth-th
    Fmagat= np.linalg.norm((Fatrx,Fatry))
    #print ('Fatx, Fatry, Fatrth',Fatrx,Fatry,(Fatrth)*180/np.pi )
    return  Fatrx, Fatry , Fmagat, Fatrth, euclD

In [13]:
def get_Speed(Ftotx,Ftoty,Ftotth):
    speed=Twist()
    if( abs(Ftotth) < .1) :
        
        speed.linear.x=0.13
        #print('lin')
        speed.angular.z=0

    else:
        
        if Ftotth < 0:                            
                if (abs( Ftotth ) < np.pi/2):
                    speed.linear.x= 0.025
                    speed.angular.z=-0.25
                    #print('open curve')
                
                else:                    
                    #print('Vang-')
                    speed.linear.x=  0.0
                    speed.angular.z=-0.5
                        
        if Ftotth > 0:
                if (abs( Ftotth ) < np.pi/2):
                    speed.linear.x= 0.05
                    speed.angular.z=.25
                    #print('open curve')
                
                else:
                    #print('Vang+')
                    speed.linear.x=  0.0
                    speed.angular.z= 0.5
    return speed

In [31]:
get_Speed(Ftotx,Ftoty,Ftotth)

linear: 
  x: 0.13
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0

In [16]:
rate = rospy.Rate(10)
counter=0
me_muero = False
xcl = 0
ycl = 0
inicio = True
n_s = 500
while not rospy.is_shutdown():
        counter+=1
        print(counter)
        Fx,Fy,Fmag,Fth          =get_Rep_Force()
        Fatx,Faty,Fmagat,Fatrth, euclD =get_Att_Force()
        #######TUNABLE
        if counter < n_s or inicio:
             signer = -10
        else:
             signer = 1
        if inicio:
             if counter > n_s:
                  inicio = False
        Ftotx= Fmag*np.cos(Fth)*.004   +    Fmagat*np.cos(Fatrth) * signer
        Ftoty= Fmag*np.sin(Fth)*.004    +    Fmagat*np.sin(Fatrth) * signer
        Ftotth=np.arctan2(Ftoty,Ftotx)
        ###ANGLE NORMALIZATION
        if ( Ftotth> np.pi ):     Ftotth=       -np.pi-    (Ftotth-np.pi)
        if (Ftotth < -np.pi): Ftotth= (Ftotth     +2 *np.pi)
        if counter==1000000:
            print('Ftotxy',Ftotx,Ftoty,Ftotth*180/np.pi, 'eulcD', euclD)
            counter=0
        ####
        speed=get_Speed(Ftotx,Ftoty,Ftotth)
        #if eculD < 2: speed.linear.x=0.2
        xy,xycl=np.array((x,y)) ,   np.array((xcl,ycl))
        euclD=np.linalg.norm(xy-xycl)
        radioRobot = 0.1
        if euclD < radioRobot * 2: 
            #speed.linear.x = 0 
            #speed.angular.z = 0
            print("im here", euclD,(euclD < radioRobot))
            #me_muero = True
            if counter > n_s:
                inicio = False
                counter = 0
            
            #rospy.sleep(1)
            #break
        pub.publish(speed)
        #if me_muero:
         #   break
        
        rate.sleep()

1
im here 0.1770414952997858 False
2
im here 0.17769695965526913 False
3
im here 0.17968252774769233 False
4
im here 0.18095943813742935 False
5
im here 0.18303915325379758 False
6
im here 0.1851528455775947 False
7
im here 0.18755092443123245 False
8
im here 0.19019315346058258 False
9
im here 0.19301304732433294 False
10
im here 0.19600204393028303 False
11
im here 0.19809482815811355 False
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184


KeyboardInterrupt: 

In [ ]:
# ENTREGAR

# TAREA POR EQUIPOS

# Programar al menos 3 conductas para el turltebot. 
# UTILIZAR MAQUINAS DE ESTADOS  y SMACH
 
 ## 1.   De ellas será  una evasión de obstáculos modificada de la tarea 1 para esta vez "rodear" el obstáculo
 (I.A. Clásica)
 
 ## 2.  Implemente navegación reactiva utilizando campos potenciales y SLAM
 (Completamente Reactivo).
 
 ## 3.  Implemente un "árbtiro" que sea capaz de detectar que el agente se encuentra en un mínimo local y repcuperar la movilidad.(Híbrido)
 
 ## 4. Utililzando tf2_ros  Obtenga un estimado de las coordenadas más "seguras".
 ### se sabe que las patas de las  mesas un son punto particularmente peligroso...
 
 Publique una transformada (tf ) en posiciones cuyas lecturas de lidar permiten suponer la presencia de una mesa o silla ... patas .
 
 
 
 
 
 
 

# Obtener la ubicación del agente en el mapa... MAPAS SLAM:... 

In [ ]:
import tf2_ros
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)
def get_coords ():
    trans = tfBuffer.lookup_transform('map', 'base_link', rospy.Time())
    return trans


In [39]:
t=get_coords()
type (t)

geometry_msgs.msg._TransformStamped.TransformStamped

In [40]:
broadcaster = tf2_ros.StaticTransformBroadcaster()

In [42]:
t.child_frame_id='here'

In [44]:
broadcaster = tf2_ros.StaticTransformBroadcaster()


In [45]:
broadcaster.sendTransform(t)